___

In [1]:
# Import libraries to be used

# Warning messages display
## import warnings
## warnings.filterwarnings(action='ignore') # https://docs.python.org/3/library/warnings.html#the-warnings-filter

# Directories/Files management
import os.path
## from zipfile import ZipFile # De momento no ha hecho falta 

# Timing
import time

# Data analysis and wrangling
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) # Show all columns in DataFrames
pd.set_option('display.max_rows', 100) # If too high, it greatly slows down the output display and freezes the kernel


# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot') # choose a style: 'plt.style.available'
sns.set_theme(context='notebook',
              style="darkgrid") # {darkgrid, whitegrid, dark, white, ticks}
palette = sns.color_palette("flare", as_cmap=True);
import altair as alt

# Machine Learning
## from sklearn.[...] import ...

In [2]:
%load_ext memory_profiler

In [3]:
t0 = time.perf_counter() 

In [4]:
# Detect Operating System running and manage paths accordingly

if os.name == 'nt': # Windows
    root = r"C:\Users\turge\CompartidoVM\0.TFM"
    print("Running on Windows.")
elif os.name == 'posix': # Ubuntu
    root = "/home/dsc/shared/0.TFM"
    print("Running on Ubuntu.")
print("root path\t", root)

Running on Windows.
root path	 C:\Users\turge\CompartidoVM\0.TFM


___

In [5]:
cols = [
    
### -----  < X > (PRE-FLIGHT DATA) -----

# Time Period
#  'YEAR', # Disregarded: for the time being, analysis limted to 2019
#  'QUARTER', # Disregarded: redundant
 'MONTH',
 'DAY_OF_MONTH',
 'DAY_OF_WEEK',
#  'FL_DATE', # Disregarded: redundant
# Airline / Aircraft
 'OP_UNIQUE_CARRIER',
#  'OP_CARRIER_AIRLINE_ID', # Disregarded: redundant
#  'OP_CARRIER', # Disregarded: redundant
 'TAIL_NUM',
#  'OP_CARRIER_FL_NUM', # Unknown in advance?
# Origin
#  'ORIGIN_AIRPORT_ID', # Disregarded: redundant
#  'ORIGIN_AIRPORT_SEQ_ID', # Disregarded: redundant
 'ORIGIN_CITY_MARKET_ID',
 'ORIGIN',
#  'ORIGIN_CITY_NAME', # Disregarded: redundant
#  'ORIGIN_STATE_ABR', # Disregarded: redundant
#  'ORIGIN_STATE_FIPS', # Federal Information Processing Standards # Not used for the moment
#  'ORIGIN_STATE_NM', # Disregarded: redundant
#  'ORIGIN_WAC', # World Area Code # Not used for the moment
# Destination
#  'DEST_AIRPORT_ID', # Disregarded: redundant
#  'DEST_AIRPORT_SEQ_ID', # Disregarded: redundant
 'DEST_CITY_MARKET_ID',
 'DEST',
#  'DEST_CITY_NAME', # Disregarded: redundant
#  'DEST_STATE_ABR', # Disregarded: redundant
#  'DEST_STATE_FIPS', # Federal Information Processing Standards # Not used for the moment
#  'DEST_STATE_NM', # Disregarded: redundant
#  'DEST_WAC', # World Area Code # Not used for the moment
# Departure Performance
 'CRS_DEP_TIME',
#  'TAXI_OUT_median', #  Output / However, the median for each airport could be used as input !! (explanation below)   
# Arrival Performance
 'CRS_ARR_TIME',
#  'TAXI_IN_median', #  Output / However, the median for each airport could be used as input !! (explanation below) 
# Flight Summaries
 'CRS_ELAPSED_TIME',
 'FLIGHTS',
 'DISTANCE',
 'DISTANCE_GROUP',

### ----- < y > (PRE-FLIGHT DATA) -----

# Departure Performance
#  'DEP_TIME', # Disregarded: redundant
#  'DEP_DELAY', # Disregarded: other potentially useful target
#  'DEP_DELAY_NEW', # Disregarded: redundant
#  'DEP_DEL15', # Disregarded: other potentially useful target
#  'DEP_DELAY_GROUP', # Disregarded: not relevant for this particular analysis
#  'DEP_TIME_BLK', # Disregarded: redundant
#  'TAXI_OUT', #  Output / However, the median for each airport could be used as input !! (explanation below)
#  'WHEELS_OFF', # Disregarded: redundant
# Arrival Performance
#  'WHEELS_ON', # Disregarded: redundant
#  'TAXI_IN', #  Output / However, the median for each airport could be used as input !! (explanation below)
#  'ARR_TIME', # Disregarded: redundant
#  'ARR_DELAY', # -------------------------------------------> MAIN TARGET !! (i.e. < y >)
#  'ARR_DELAY_NEW', # Disregarded: redundant
 'ARR_DEL15', # Disregarded: other potentially useful target
#  'ARR_DELAY_GROUP', # Disregarded: not relevant for this particular analysis
#  'ARR_TIME_BLK', # Disregarded: redundant
# Cancellations and Diversions
#  'CANCELLED', # Disregarded: not relevant for this particular analysis
#  'CANCELLATION_CODE', # Disregarded: not relevant for this particular analysis
#  'DIVERTED', # Disregarded: not relevant for this particular analysis
# Flight Summaries
#  'ACTUAL_ELAPSED_TIME', # Disregarded: redundant
#  'AIR_TIME', # Disregarded: redundant
# Cause of Delay
#  'CARRIER_DELAY', # Disregarded: other potentially useful target
#  'WEATHER_DELAY', # Disregarded: other potentially useful target
#  'NAS_DELAY', # Disregarded: other potentially useful target
#  'SECURITY_DELAY', # Disregarded: other potentially useful target
#  'LATE_AIRCRAFT_DELAY', # Disregarded: other potentially useful target
# Gate Return Information at Origin Airport (Data starts 10/2008)
#  'FIRST_DEP_TIME', # Disregarded: not relevant for this particular analysis
#  'TOTAL_ADD_GTIME', # Disregarded: not relevant for this particular analysis
#  'LONGEST_ADD_GTIME', # Disregarded: not relevant for this particular analysis
# Diverted Airport Information (Data starts 10/2008)
#  'DIV_AIRPORT_LANDINGS', # Disregarded: not relevant for this particular analysis
]

In [6]:
cols

['MONTH',
 'DAY_OF_MONTH',
 'DAY_OF_WEEK',
 'OP_UNIQUE_CARRIER',
 'TAIL_NUM',
 'ORIGIN_CITY_MARKET_ID',
 'ORIGIN',
 'DEST_CITY_MARKET_ID',
 'DEST',
 'CRS_DEP_TIME',
 'CRS_ARR_TIME',
 'CRS_ELAPSED_TIME',
 'FLIGHTS',
 'DISTANCE',
 'DISTANCE_GROUP',
 'ARR_DEL15']

In [7]:
preprocessed_input_csv_path = os.path.join(root,
                                           "Output_Data",
                                           "US_DoT",
                                           "AL_OTP_MVP_Preprocessed_19_v2_clean.csv")
preprocessed_input_csv_path

'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Output_Data\\US_DoT\\AL_OTP_MVP_Preprocessed_19_v2_clean.csv'

In [8]:
%%time

df = pd.read_csv(preprocessed_input_csv_path,
                 encoding='latin1',
#                  nrows=1e4,
                 usecols=cols, # This way, the extra column is disregarded for the loading process
                 low_memory = False)

Wall time: 1min 34s


In [9]:
df.shape

(7268232, 16)

In [10]:
df = df.sample(int(1e3))

In [11]:
df

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,ORIGIN_CITY_MARKET_ID,ORIGIN,DEST_CITY_MARKET_ID,DEST,CRS_DEP_TIME,CRS_ARR_TIME,ARR_DEL15,CRS_ELAPSED_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP
5456320,10,11,5,UA,N66828,31703,EWR,32467,MIA,1955,2257,0.0,182.0,1.0,1085.0,5
582185,2,18,1,WN,N562WN,32457,SJC,32575,BUR,1915,2025,0.0,70.0,1.0,296.0,2
5846651,10,6,7,OO,N194SY,32457,SJC,30713,BOI,1925,2215,0.0,110.0,1.0,523.0,3
2018750,4,18,4,WN,N932WN,30140,ABQ,33198,MCI,1650,1935,1.0,105.0,1.0,718.0,3
1536298,3,2,6,DL,N972DL,30928,ICT,30397,ATL,1400,1708,0.0,128.0,1.0,782.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2954182,6,1,6,OO,N938SW,34236,SBP,32457,SFO,1615,1728,0.0,73.0,1.0,190.0,1
6135856,11,4,1,UA,N814UA,33214,SAT,30852,IAD,1708,2111,0.0,183.0,1.0,1362.0,6
306672,1,13,7,B6,N584JB,30559,SEA,32575,LGB,600,837,0.0,157.0,1.0,965.0,4
991496,2,17,7,EV,N14570,32884,LAN,30977,ORD,645,703,0.0,78.0,1.0,179.0,1


In [12]:
df.dtypes

MONTH                      int64
DAY_OF_MONTH               int64
DAY_OF_WEEK                int64
OP_UNIQUE_CARRIER         object
TAIL_NUM                  object
ORIGIN_CITY_MARKET_ID      int64
ORIGIN                    object
DEST_CITY_MARKET_ID        int64
DEST                      object
CRS_DEP_TIME               int64
CRS_ARR_TIME               int64
ARR_DEL15                float64
CRS_ELAPSED_TIME         float64
FLIGHTS                  float64
DISTANCE                 float64
DISTANCE_GROUP             int64
dtype: object

### Feature analysis

[Dealing with categorical features with high cardinality: Target Encoding](https://medium.com/@kr.vishwesh54/dealing-with-categorical-features-with-high-cardinality-target-encoding-baa9298bf257)

In [ ]:
X = df.drop('ARR_DEL15', axis=1)
X.sample(5)

In [ ]:
y = df['ARR_DEL15']
y.sample(5)

___

In [ ]:
%memit

### A Simple Guide to Scikit-learn Pipelines

#### Transformers

Before building the pipeline, let's split the data into a train and test set so that the performance of the model can be validated.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

1. The first step in building the pipeline is to define each transformer type.

In [ ]:
%%time

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

2. Next, let's use the ColumnTransformer to apply the transformations to the correct columns in the dataframe. Before building this, the numeric and categorical columns shall be listed.

In [ ]:
%%time

numeric_features = df.select_dtypes(include=['int64', 'float64']).drop(['ARR_DEL15'], axis=1).columns
categorical_features = df.select_dtypes(include=['object']).columns
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

#### Fitting the classifier

The next step is to create a pipeline that combines the preprocessor created above with a classifier. In this case a simple RandomForestClassifier has been used to start with.

In [ ]:
%%time

from sklearn.ensemble import RandomForestClassifier
rf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(random_state=0))])
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

#### Model selection

**A pipeline can also be used during the model selection process**. The following example code loops through a number of scikit-learn classifiers applying the transformations and training the model.

In [ ]:
%%time

from sklearn.metrics import accuracy_score, log_loss, f1_score, recall_score, precision_score, roc_auc_score, roc_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True, random_state=0),
#     NuSVC(probability=True), # ValueError: b'specified nu is infeasible'
    DecisionTreeClassifier(random_state=0),
    RandomForestClassifier(random_state=0),
    AdaBoostClassifier(random_state=0),
    GradientBoostingClassifier(random_state=0)
    ]
for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    pipe.fit(X_train, y_train)   
    print(classifier)
    
    # TEST !!!     
    predictions = pipe.predict(X_test)
    print(f1_score(y_test, predictions), recall_score(y_test, predictions), precision_score(y_test, predictions), accuracy_score(y_test, predictions))
    
    print("model score: %.3f \n" % pipe.score(X_test, y_test))

**The pipeline can also be used in grid search** to find the best performing parameters. To do this, let's first create a parameter grid for the chosen model.

*One important thing to note is that there is a need to append the name given to the classifier part of the pipeline to each parameter name. In the code above its name is ‘classifier’ so 'classifier__' has been added to each parameter.*

Next a grid search object has been created, which includes the original pipeline. When fit is called, the transformations are applied to the data, before a cross-validated grid-search is performed over the parameter grid.

In [ ]:
import sklearn
sklearn.metrics.SCORERS.keys()

In [ ]:
%%time

param_grid = { 
    'classifier__n_estimators': [200, 500],
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__max_depth' : [4,5,6,7,8],
    'classifier__criterion' :['gini', 'entropy']}
from sklearn.model_selection import GridSearchCV
# Setting refit='AUC', refits an estimator on the whole dataset with the
# parameter setting that has the best cross-validated AUC score.
# That estimator is made available at ``gs.best_estimator_`` along with
# parameters like ``gs.best_score_``, ``gs.best_params_`` and
# ``gs.best_index_``
scoring = {'AUC': 'roc_auc', 'F1': 'f1', 'Precision': 'precision',
           'Recall': 'recall', 'Accuracy': 'accuracy'}
CV = GridSearchCV(rf, param_grid, n_jobs= 1, scoring=scoring, refit='AUC') # change the scorer metric
                  
CV.fit(X_train, y_train)  
print(CV.best_params_)    
print(CV.best_score_)

In [ ]:
predictions = CV.predict(X_test)
probabilities = CV.predict_proba(X_test)

### [Classification Metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics)

- Binary classification (only):
    - [precision_recall_curve](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_curve.html#sklearn.metrics.precision_recall_curve): Compute precision-recall pairs for different probability thresholds.
        - The precision is the ratio `tp / (tp + fp)` where `tp` is the number of true positives and `fp` the number of false positives. The **precision** is intuitively the ability of the classifier **not to label as positive a sample that is negative**.
        - The recall is the ratio `tp / (tp + fn)` where `tp` is the number of true positives and `fn` the number of false negatives. The **recall** is intuitively the ability of the classifier to **find all the positive samples**.

    ### RECALL could be a potentially strong metric for this case; "from all the flights classified as delayed, the actual (true) number delyed flights is as high as possible."

    - [roc_curve](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html#sklearn.metrics.roc_curve): A receiver operating characteristic (ROC), or simply ROC curve, is a graphical plot which illustrates the performance of a binary classifier system as its discrimination threshold is varied. It is created by plotting the fraction of true positives out of the positives (TPR = true positive rate) vs. the fraction of false positives out of the negatives (FPR = false positive rate), at various threshold settings. TPR is also known as sensitivity, and FPR is one minus the specificity or true negative rate.
        
- Multi-class classification (or binary):
    - [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix): Compute confusion matrix to evaluate the accuracy of a classification.
    ![IMG_Confusion_Matrix](https://scikit-learn.org/stable/_images/sphx_glr_plot_confusion_matrix_0011.png)
    - [roc_auc_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score): Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.

- Multi-label classification (or binary or multi-class):
    - [accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score): Accuracy classification score.
        - It is the ratio of number of correct predictions to the total number of input samples.
        - **It works well only if there are equal number of samples belonging to each class.**
    - [classification_report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report): Build a text report showing the main classification metrics.
    - [f1_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score): Compute the F1 score, also known as balanced F-score or F-measure.
        - The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal: `F1 = 2 * (precision * recall) / (precision + recall)`
    - [precision_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score)
    - [recall_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score)

### Which metric should be used then?
##### Bear in mind that it's a clear case of imbalanced data

Nomenclature:
- Delayed = Positive
- On-time = Negative 

Considering this:
- False positive (Type I error) → Wrongly classifying an On-time flight as a Delayed flight → Not significantly relevant
- **False negative (Type II error) → Wrongly classifying a Delayed flight as an On-time flight → Highly relevant**

**F-beta** score (\$ F_\beta \$):
![F-beta score](https://wikimedia.org/api/rest_v1/media/math/render/svg/136f45612c08805f4254f63d2f2524bc25075fff)

Two commonly used values for β are:
- **2 : weighs recall higher than precision**
- 0.5 : weighs recall lower than precision.


*Probably most people in the industry would accept that an **OTP of 80%* or above is pretty good**. That’s 4 in 5 flights arriving within 15 minutes of their scheduled arrival time. The very best airlines and airports succeed in punctuality closer to 90% - but they remain the exception rather than the rule.* (Source: [OAG](https://www.oag.com/on-time-performance-airlines-airports))

The actual data from the 7268232 records comprising the OTP dataset accurately confirm this hypothesis:
```
Delays: 5878979 (80.89%)
On-time: 1389253 (19.11%)
```

In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score, roc_auc_score, roc_curve, accuracy_score

tpr, fpr, thresolds = roc_curve(y_test, probabilities[:, 1])
plt.plot(tpr, fpr)
f1_score(y_test, predictions), recall_score(y_test, predictions), precision_score(y_test, predictions), accuracy_score(y_test, predictions)